# **4 – Model Developmemt**

## Objectives
* Build and train a CNN to classify cherry leaf images as either healthy or diseased
* Evaluate model performance and save it for deployment

## Inputs
* Train/val/test folders from inputs/dataset

## Outputs
- Trained CNN model
- Accuracy/loss plots
- Evaluation results on test set


---

# Change working directory

Change the working directory from its current folder to its parent folder

In [3]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\amyno\\OneDrive\\Documents\\CherryLeafProject\\milestone-project-mildew-detection-in-cherry-leaves\\jupyter_notebooks'

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [4]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [5]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\amyno\\OneDrive\\Documents\\CherryLeafProject\\milestone-project-mildew-detection-in-cherry-leaves'

---

# Load image datasets

Load the preprocessed cherry leaf images from the inputs/dataset directory using TensorFlow.

TensorFlow's image-data-set-from-directory function will efficiently feed the data into the model in batches, while also keeping the images labelled and shuffled automatically

Install tensorflow

In [6]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Import tensorflow's image_dataset_from_directory

In [7]:
from tensorflow.keras.utils import image_dataset_from_directory

Determine batch size and image size

In [8]:
img_size = (256, 256)
batch_size = 32

The most common batch size (how many images the model will process at one time) is 32. This is a sensible number for this project. Whilst it may take longer, the model will be more accurate and less susceptable to overfitting

The image size was checked during the data prep notebook, but it is a good idea to put extra safeguards in place 

Load training, validation and test datasets from inputs directory

In [ ]:
train_ds = image_dataset_from_directory( #code helpfully inspired by Keras (ref. in readme)
    "inputs/dataset/train",
    labels="inferred",
    label_mode="binary", 
    image_size=img_size,
    batch_size=batch_size,
    seed=123
)

Found 2944 files belonging to 2 classes.


In [10]:
val_ds = image_dataset_from_directory(
    "inputs/dataset/val",
    labels="inferred",
    label_mode="binary",
    image_size=img_size,
    batch_size=batch_size,
    seed=123
)

Found 632 files belonging to 2 classes.


In [11]:
test_ds = image_dataset_from_directory(
    "inputs/dataset/test",
    labels="inferred",
    label_mode="binary",
    image_size=img_size,
    batch_size=batch_size,
    seed=123
)

Found 632 files belonging to 2 classes.


All of the datasets are loaded and are showing the expected number of images in each, with the 2 classes referring to the healthy and powdery mildew classes

---

### Credits
* TensorFlow tutorial
* Keras documentation

---